# Import Libraries

In [ ]:
from pypdf import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer

# Read Document

In [ ]:
# Load pdf documents
documents_1 = ''

reader = PdfReader('/workspaces/digital_research_guide/1706.03762v7.pdf')
for page in reader.pages:
    documents_1 += page.extract_text()

In [ ]:
print(documents_1)

In [ ]:
type(documents_1)

In [ ]:
cleaned_string = documents_1.replace('\n', ' ')
cleaned_string

In [ ]:
print(cleaned_string)

In [ ]:
sentence_list = cleaned_string.split('. ')

In [ ]:
len(sentence_list)

In [ ]:
context_list = []

In [ ]:
group_size = 20
overlap = 5
i = 0 
while True:
    end_index = min(len(sentence_list),(i+group_size))
    group = sentence_list[i:i+group_size]
    text = '. '.join(group)
    context_list.append(text)
    i+=group_size-overlap
    if i>=len(sentence_list):
        break


In [ ]:
len(context_list)

In [ ]:
context_list[0]

# Create Embedding

In [ ]:
check_point = 'paraphrase-MiniLM-L6-v2'
check_point = 'mixedbread-ai/mxbai-embed-large-v1'

In [ ]:
model = SentenceTransformer(check_point)

In [ ]:
model.get_max_seq_length()

In [ ]:
model.get_sentence_embedding_dimension()

In [ ]:
def get_embeddings(doc):
    model_input = doc
    out =  model.encode(model_input)
    return out

In [ ]:
x = list(map(get_embeddings,context_list))

In [ ]:
import numpy as np
x_array = np.array(x)

In [ ]:
x_array.shape

[Indexes](https://github.com/facebookresearch/faiss/wiki/Faiss-indexes)

In [ ]:
import faiss
index = faiss.IndexFlatL2(model.get_sentence_embedding_dimension())
index.add(x_array)

In [ ]:
faiss.write_index(index, 'test.index')

In [ ]:
query = "What is Multi Head Attention"
query_embedding = model.encode(query).reshape(1, -1)

In [ ]:
top_k = 10
_scores, binary_ids = index.search(query_embedding, top_k)
binary_ids = binary_ids[0]
_scores = _scores[0]

In [ ]:
_scores, binary_ids

In [ ]:
for idx in binary_ids:
    print(idx)
    doc = context_list[idx]
    print(doc)
    print("----------------------------------------------------------------------")
    print()